In [1]:
# Import Required Libraries
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd
import datetime
import numpy as np
import os
import talib

In [2]:
# Install TA-Lib if not available
try:
    import talib
    print("✓ TA-Lib is available")
except ImportError:
    print("Installing TA-Lib...")
    import subprocess
    import sys
    
    # Try to install TA-Lib
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "TA-Lib"])
        import talib
        print("✓ TA-Lib installed successfully")
    except Exception as e:
        print(f"❌ Failed to install TA-Lib: {e}")
        print("💡 Please install TA-Lib manually:")
        print("   conda install -c conda-forge ta-lib")
        print("   or download from: https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib")

✓ TA-Lib is available


In [3]:
long_conditions = {
        'bullish_reversal_patterns': [
            'CDLHAMMER',              # Hammer
            'CDLINVERTEDHAMMER',      # Inverted Hammer
            'CDLMORNINGSTAR',         # Morning Star
            'CDLMORNINGDOJISTAR',     # Morning Doji Star
            'CDLENGULFING',           # Bullish Engulfing
            'CDLPIERCING',            # Piercing Pattern
            'CDLHARAMI',              # Bullish Harami
            'CDLHARAMICROSS',         # Bullish Harami Cross
            'CDLTAKURI',              # Takuri (Dragonfly Doji)
        ],
        
        'bullish_continuation_patterns': [
            'CDL3WHITESOLDIERS',      # Three White Soldiers
            'CDLRISEFALL3METHODS',    # Rising Three Methods
            'CDLMATHOLD',             # Mat Hold
            'CDLSEPARATINGLINES',     # Bullish Separating Lines
            'CDLTASUKIGAP',           # Bullish Tasuki Gap uptrend
        ],
        
        'bullish_bottom_patterns': [
            'CDLABANDONEDBABY',       # Abandoned Baby
            'CDLLADDERBOTTOM',        # Ladder Bottom
            'CDLMATCHINGLOW',         # Matching Low
            'CDLUNIQUE3RIVER',        # Unique Three River
        ],
        
        'bullish_special_patterns': [
            'CDL3INSIDE',             # Three Inside Up
            'CDL3OUTSIDE',            # Three Outside Up
            'CDLBELTHOLD',            # Belt Hold
            'CDLBREAKAWAY',           # Breakaway
            'CDLKICKING',             # Kicking
            'CDLKICKINGBYLENGTH',     # Kicking By Length
            'CDLSTICKSANDWICH',       # Stick Sandwich
        ]
    }

In [ ]:
# sell_conditions = {
#         'bearish_reversal_patterns': [
#             'CDLSHOOTINGSTAR',        # Shooting Star
#             'CDLHANGINGMAN',          # Hanging Man
#             'CDLEVENINGSTAR',         # Evening Star
#             'CDLEVENINGDOJISTAR',     # Evening Doji Star
#             'CDLENGULFING',           # Bearish Engulfing (negative signal)
#             'CDLDARKCLOUDCOVER',      # Dark Cloud Cover
#             'CDLGRAVESTONEDOJI',      # Gravestone Doji
#             'CDLHARAMI',              # Bearish Harami (negative signal)
#             'CDLHARAMICROSS',         # Bearish Harami Cross (negative signal)
#         ],
        
#         'bearish_continuation_patterns': [
#             'CDL3BLACKCROWS',         # Three Black Crows
#             'CDLFALL3METHODS',        # Falling Three Methods
#             'CDLIDENTICAL3CROWS',     # Identical Three Crows
#             'CDLADVANCEBLOCK',        # Advance Block
#             'CDLSTALLEDPATTERN',      # Stalled Pattern
#         ],
        
#         'bearish_top_patterns': [
#             'CDL2CROWS',              # Two Crows
#             'CDLUPSIDEGAP2CROWS',     # Upside Gap Two Crows
#             'CDLXSIDEGAP3METHODS',    # Upside/Downside Gap Three Methods
#         ],
        
#         'bearish_special_patterns': [
#             'CDL3INSIDE',             # Three Inside Down
#             'CDL3OUTSIDE',            # Three Outside Down
#             'CDL3LINESTRIKE',         # Three Line Strike (bearish context)
#             'CDLINNECK',              # In-Neck Line
#             'CDLONNECK',              # On-Neck Line
#             'CDLTHRUSTING',           # Thrusting Pattern
#             'CDLBELTHOLD',            # Belt Hold (bearish context)
#             'CDLBREAKAWAY',           # Breakaway (bearish context)
#         ]
#     }

In [ ]:
# Strategy Candlestick Patterns - Practical Implementation

class Long_Candlesticks(bt.Strategy):
    """
    Strategy focusing on long candlestick patterns for trade entries
    Entry signals based on bullish patterns using TA-Lib candlestick functions
    
    Stop-loss and take-profit based on ATR

    Ideally suited for swing trading with signals on 1H, 4H charts

    """
    
    params = (
        ('printlog', False),
        ('atr_period', 14),
        ('atr_sl_multiplier', 2.0),
        ('atr_tp_multiplier', 3.0),
        ('fixed_atr', 40),  # Fixed ATR value in points

        # to check here
        ('lookback_period', 50),  # Number of bars to look back for pattern detection
    )

    def __init__(self):
        # ATR for stop-loss and take-profit calculations
        self.atr = self.p.fixed_atr

        self.stop_loss = None
        self.take_profit = None
        
        self.order = None
        self.trade_count = 0
        
        # Store OHLC data for TA-Lib calculations
        self.open_data = []
        self.high_data = []
        self.low_data = []
        self.close_data = []

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}')
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.trade_count += 1
        self.log(f'TRADE #{self.trade_count}: PROFIT {trade.pnl:.2f}', doprint=True)

    def check_bullish_patterns(self, open_prices, high_prices, low_prices, close_prices):
        """Check for bullish candlestick patterns using TA-Lib"""
        bullish_signals = []
        
        try:
            # Convert to numpy arrays
            open_arr = np.array(open_prices, dtype=np.float64)
            high_arr = np.array(high_prices, dtype=np.float64)
            low_arr = np.array(low_prices, dtype=np.float64)
            close_arr = np.array(close_prices, dtype=np.float64)
            
            # Check bullish reversal patterns
            hammer = talib.CDLHAMMER(open_arr, high_arr, low_arr, close_arr)
            inverted_hammer = talib.CDLINVERTEDHAMMER(open_arr, high_arr, low_arr, close_arr)
            morning_star = talib.CDLMORNINGSTAR(open_arr, high_arr, low_arr, close_arr)
            morning_doji = talib.CDLMORNINGDOJISTAR(open_arr, high_arr, low_arr, close_arr)
            engulfing = talib.CDLENGULFING(open_arr, high_arr, low_arr, close_arr)
            piercing = talib.CDLPIERCING(open_arr, high_arr, low_arr, close_arr)
            harami = talib.CDLHARAMI(open_arr, high_arr, low_arr, close_arr)

            # added patterns
            takuri = talib.CDLTAKURI(open_arr, high_arr, low_arr, close_arr)  # Takuri (Dragonfly Doji)
            rise_fall = talib.CDLRISEFALL3METHODS(open_arr, high_arr, low_arr, close_arr)  # Rising Three Methods
            mat_hold = talib.CDLMATHOLD(open_arr, high_arr, low_arr, close_arr)  # Mat Hold
            separating_lines = talib.CDLSEPARATINGLINES(open_arr, high_arr, low_arr, close_arr)  # Bullish Separating Lines
            tasuki_gap = talib.CDLTASUKIGAP(open_arr, high_arr, low_arr, close_arr)  # Bullish Tasuki Gap uptrend
            abandoned_baby = talib.CDLABANDONEDBABY(open_arr, high_arr, low_arr, close_arr)  # Abandoned Baby
            ladder_bottom = talib.CDLLADDERBOTTOM(open_arr, high_arr, low_arr, close_arr)  # Ladder Bottom
            matching_low = talib.CDLMATCHINGLOW(open_arr, high_arr, low_arr, close_arr)  # Matching Low
            unique_three_river = talib.CDLUNIQUE3RIVER(open_arr, high_arr, low_arr, close_arr)  # Unique Three River
            three_inside = talib.CDL3INSIDE(open_arr, high_arr, low_arr, close_arr)  # Three Inside Up
            three_outside = talib.CDL3OUTSIDE(open_arr, high_arr, low_arr, close_arr)  # Three Outside Up
            belt_hold = talib.CDLBELTHOLD(open_arr, high_arr, low_arr, close_arr)  # Belt Hold
            breakaway = talib.CDLBREAKAWAY(open_arr, high_arr, low_arr, close_arr)  # Breakaway
            kicking = talib.CDLKICKING(open_arr, high_arr, low_arr, close_arr)  # Kicking
            kicking_by_length = talib.CDLKICKINGBYLENGTH(open_arr, high_arr, low_arr, close_arr)  # Kicking By Length
            sticks_sandwich = talib.CDLSTICKSANDWICH(open_arr, high_arr, low_arr, close_arr)  # Stick Sandwich

            # Check bullish continuation patterns
            three_white_soldiers = talib.CDL3WHITESOLDIERS(open_arr, high_arr, low_arr, close_arr)
            
            # Combine all bullish signals (positive values indicate bullish patterns)
            patterns = [hammer, inverted_hammer, morning_star, morning_doji, 
                       engulfing, piercing, harami, three_white_soldiers, 
                       takuri, rise_fall, mat_hold, separating_lines, tasuki_gap, 
                       abandoned_baby, ladder_bottom, matching_low, unique_three_river, 
                       three_inside, three_outside, belt_hold, breakaway, kicking, 
                       kicking_by_length, sticks_sandwich]

            # Check if any pattern shows a bullish signal (> 0) in the last bar
            for pattern in patterns:
                if len(pattern) > 0 and pattern[-1] > 0:
                    bullish_signals.append(pattern[-1])
            
            return len(bullish_signals) > 0
            
        except Exception as e:
            self.log(f"Pattern detection error: {e}")
            return False

    def next(self):
        # Store current OHLC data
        self.open_data.append(float(self.data.open[0]))
        self.high_data.append(float(self.data.high[0]))
        self.low_data.append(float(self.data.low[0]))
        self.close_data.append(float(self.data.close[0]))
        
        # Keep only the last lookback_period bars
        if len(self.close_data) > self.p.lookback_period:
            self.open_data = self.open_data[-self.p.lookback_period:]
            self.high_data = self.high_data[-self.p.lookback_period:]
            self.low_data = self.low_data[-self.p.lookback_period:]
            self.close_data = self.close_data[-self.p.lookback_period:]
        # Wait for enough data for pattern detection
        # Wait for enough data and ATR to stabilize
        if len(self.close_data) < 30 or len(self.data) < 60:
            return

        if self.order:
            return

        if not self.position:
            # Check for bullish candlestick patterns
            if self.check_bullish_patterns(self.open_data, self.high_data, 
                                         self.low_data, self.close_data):
                self.log('BUY SIGNAL: Bullish candlestick pattern detected')
                self.order = self.buy()
                # Set stop-loss and take-profit based on fixed ATR
                self.stop_loss = self.data.close[0] - (self.atr * self.p.atr_sl_multiplier)
                self.take_profit = self.data.close[0] + (self.atr * self.p.atr_tp_multiplier)
                self.take_profit = self.data.close[0] + (self.atr[0] * self.p.atr_tp_multiplier)
                self.log(f'SET SL: {self.stop_loss:.2f}, TP: {self.take_profit:.2f}')
        else:
            # Manage existing position with ATR-based stop-loss and take-profit
            if self.data.close[0] <= self.stop_loss:
                self.log('STOP-LOSS HIT')
                self.order = self.sell()
            elif self.data.close[0] >= self.take_profit:
                self.log('TAKE-PROFIT HIT')
                self.order = self.sell()

    def stop(self):
        self.log(f'Long_Candlesticks Strategy: {self.trade_count} trades, Final Value: ${self.broker.getvalue():.2f}', doprint=True)

print("✓ Long_Candlesticks Strategy created - using TA-Lib candlestick patterns!")

✓ Long_Candlesticks Strategy created - using TA-Lib candlestick patterns!


In [ ]:
# Backtest Function for Long_Candlesticks Strategy

def test_long_candlesticks_strategy():
    """Test the Long_Candlesticks strategy with S&P 500 data"""
    print("=== TESTING Long_Candlesticks ===\n")
    
    cerebro = bt.Cerebro()
    
    # Load S&P 500 data
    spy_file = r'../10_backtrader\01_CDL_Patterns_Strategy\01_data\SP500_generating_synthetic_financial_return_series\SP500_bootstrap_prices.csv'

    df = pd.read_csv(spy_file, skiprows=3, header=None)
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').dropna()
    
    # Use last 20 years for testing
    end_date = df['Date'].max()
    start_date = end_date - pd.DateOffset(years=20)
    df_recent = df[df['Date'] >= start_date].copy()
    
    print(f"📊 Data: {len(df_recent)} trading days from {start_date.date()} to {end_date.date()}")
    
    # Create temporary file for backtrader
    temp_file = 'temp_simplified_test.csv'
    df_recent.to_csv(temp_file, index=False)
    
    # Create data feed
    data = btfeeds.GenericCSVData(
        dataname=temp_file,
        dtformat=('%Y-%m-%d'),
        datetime=0, open=1, high=2, low=3, close=4, volume=5,
        openinterest=-1, headers=True,
    )

    # Add data and strategy
    cerebro.adddata(data)
    cerebro.addstrategy(Long_Candlesticks, printlog=False)
    
    # Set initial capital and commission
    initial_capital = 100000.0
    cerebro.broker.setcash(initial_capital)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    print(f"💰 Starting Capital: ${initial_capital:,.2f}")
    print("🚀 Running backtest...\n")
    
    # Run backtest
    results = cerebro.run()
    
    # Clean up temporary file
    if os.path.exists(temp_file):
        os.remove(temp_file)
    
    # Calculate results
    final_value = cerebro.broker.getvalue()
    total_return = ((final_value - initial_capital) / initial_capital) * 100
    
    print(f"\n=== BACKTEST RESULTS ===")
    print(f"💰 Final Portfolio Value: ${final_value:,.2f}")
    print(f"📈 Total Return: {total_return:.2f}%")
    print(f"💵 Net Profit: ${final_value - initial_capital:,.2f}")
    
    # Analyze trades
    strat = results[0]
    trade_analysis = strat.analyzers.trades.get_analysis()
    
    if trade_analysis and hasattr(trade_analysis, 'total'):
        total_trades = getattr(trade_analysis.total, 'total', 0)
        print(f"\n=== TRADE ANALYSIS ===")
        print(f"📊 Total Trades: {total_trades}")
        
        # Winning trades
        if hasattr(trade_analysis, 'won'):
            won_trades = getattr(trade_analysis.won, 'total', 0)
            won_pnl = getattr(trade_analysis.won, 'pnl', {}).get('total', 0)
            win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
            print(f"✅ Winning Trades: {won_trades} ({win_rate:.1f}%)")
            if won_pnl:
                print(f"💰 Winning PnL: ${won_pnl:.2f}")
        
        # Losing trades  
        if hasattr(trade_analysis, 'lost'):
            lost_trades = getattr(trade_analysis.lost, 'total', 0)
            lost_pnl = getattr(trade_analysis.lost, 'pnl', {}).get('total', 0)
            print(f"❌ Losing Trades: {lost_trades}")
            if lost_pnl:
                print(f"💸 Losing PnL: ${lost_pnl:.2f}")
    
    # Performance metrics
    print(f"\n=== PERFORMANCE METRICS ===")
    
    # Sharpe Ratio
    sharpe_analysis = strat.analyzers.sharpe.get_analysis()
    if sharpe_analysis and 'sharperatio' in sharpe_analysis:
        sharpe_ratio = sharpe_analysis['sharperatio'] or 0
        print(f"📊 Sharpe Ratio: {sharpe_ratio:.4f}")

    # Drawdown Analysis
    drawdown_analysis = strat.analyzers.drawdown.get_analysis()
    if drawdown_analysis:
        max_dd = drawdown_analysis.get('max', {}).get('drawdown', 0)
        print(f"📉 Max Drawdown: {max_dd:.2f}%")
    
    return results, total_trades, final_value

print("✓ Backtest function ready for Long_Candlesticks")

✓ Backtest function ready for Long_Candlesticks


In [ ]:
# Execute Long_Candlesticks Backtest
try:
    results, trade_count, final_value = test_long_candlesticks_strategy()
    
    print(f"\n🎉 BACKTEST COMPLETED SUCCESSFULLY!")
    print(f"📊 Summary: {trade_count} trades, Final Value: ${final_value:,.2f}")
    
    if trade_count > 10:
        print("✅ Strategy shows active trading with multiple trades")
    elif trade_count > 1:
        print("📈 Strategy executed multiple trades")
    else:
        print("⚠️ Strategy was very selective with few trades")
        
except Exception as e:
    print(f"❌ Error running backtest: {str(e)}")
    print("💡 Make sure the S&P 500 data file exists at the specified path")

=== TESTING Long_Candlesticks ===

📊 Data: 5031 trading days from 2005-09-17 to 2025-09-17
💰 Starting Capital: $100,000.00
🚀 Running backtest...

2006-01-03, TRADE #1: PROFIT -24.45
2006-03-16, TRADE #2: PROFIT 34.22
2006-05-18, TRADE #3: PROFIT -32.64
2006-06-13, TRADE #4: PROFIT -30.95
2006-08-31, TRADE #5: PROFIT 48.09
2006-10-05, TRADE #6: PROFIT 38.90
2006-10-26, TRADE #7: PROFIT 32.63
2006-11-16, TRADE #8: PROFIT 29.22
2006-11-28, TRADE #9: PROFIT -21.08
2007-01-16, TRADE #10: PROFIT 31.26
2007-02-28, TRADE #11: PROFIT -41.48
2007-04-09, TRADE #12: PROFIT 48.75
2007-05-08, TRADE #13: PROFIT 40.89
2007-07-13, TRADE #14: PROFIT 40.36
2007-07-25, TRADE #15: PROFIT -38.17
2007-08-06, TRADE #16: PROFIT -40.86
2007-08-15, TRADE #17: PROFIT -71.06
2007-10-02, TRADE #18: PROFIT 101.02
2007-10-22, TRADE #19: PROFIT -48.01
2007-11-08, TRADE #20: PROFIT -40.88
2007-11-23, TRADE #21: PROFIT -49.97
2008-01-09, TRADE #22: PROFIT -50.49
2008-01-18, TRADE #23: PROFIT -77.98
2008-03-11, TRADE #24